In [ ]:
!pip install translate
!pip install googletrans==4.0.0-rc1

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
dataBasePath = "D:/PHD/Projects/data"

In [6]:
from googletrans import Translator as tr
import textwrap as txt
import pandas as pd

class TextLengthExceededError(Exception):
    def __init__(self, length, message = f"Text size is exceeded available limit of 500 characters."):
        self.length = length
        self.message = message
        super().__init__(self.message)
    def __str__(self):
        return f"{self.message} Actual size is {self.length}"


class ItalianToEnglishTranslator:
    
    def translate(self, value):
        translator = tr()
        if len(value)>500:
            raise TextLengthExceededError(len(value))
        return translator.translate(value, dest="en", src="it").text
    
    
    def translateList(self, listOfValues):
        listOfTranslatedValues = []
        for value in listOfValues:
            try:
                listOfTranslatedValues.append(self.translate(value))
            except TextLengthExceededError:
                longValueSplitted = self.__prepareValue(value)
                longValueTranslation = []
                for sentence in longValueSplitted:
                    longValueTranslation.append(self.translate(sentence))
                listOfTranslatedValues.append(longValueTranslation)
        return listOfTranslatedValues
    
    def __prepareValue(self, value, limit=-1):
        valueLength = len(value)
        if valueLength<limit:
            return [value]
        return value.split(".", maxsplit=limit)
    
    
    
class DatabaseHelper:
    
    def extractColumnsForTranslation(self, path, surveySheetName, responseSheetName, surveyColumnsLocation="A", 
                                    responseColumnsLocation=12):
        
        rawItalianColumns = pd.read_excel(path, usecols=surveyColumnsLocation, engine="openpyxl", sheet_name=surveySheetName)
        responseColumns = pd.read_excel(path, skiprows=responseColumnsLocation, nrows=1, 
                                        engine="openpyxl", sheet_name=responseSheetName)
        if 'Unnamed: 0' in responseColumns.columns:
            responseColumns.drop("Unnamed: 0", axis = 1, inplace=True)
            
        responseColumns = responseColumns.columns.to_list()
        responseColumns = [responseColumns[x].replace(",", ".",) for x in range(len(responseColumns))]

        responseColumnsValues = []

        for i in range(len(rawItalianColumns)):
            try:
                if any(responseColumns[x] in rawItalianColumns.iloc[i, 0] for x in range(len(responseColumns))):
                    responseColumnsValues.append(rawItalianColumns.iloc[i, 0])
            except TypeError:
                pass
        
        return responseColumnsValues
        
        
    
    

In [7]:
dbHelper = DatabaseHelper()
respCol = dbHelper.extractColumnsForTranslation(f"{dataBasePath}/report.xlsx", "Questionario_2781022573", "Risposte_2781022573")

translator = ItalianToEnglishTranslator()
translatedColumnNames = translator.translateList(responseColumnsValues)
translatedColumnsDataframe = pd.DataFrame(translatedColumnNames)
translatedColumnsDataframe.to_csv(f"{dataBasePath}/surveyColumnsInEnglish.csv")

In [ ]:
testTranslator = tr()
translated = testTranslator.translate(responseColumnsValues[0], dest="en", src="it")


In [ ]:
translated.src

In [ ]:
path